In [1]:
from tqdm.notebook import trange
from tqdm.notebook import tqdm
import requests
from subprocess import call
from os import listdir, mkdir
from os.path import  isfile, join
import gzip
import json

In [2]:
from Diaries import Diary
from TextUtils import TextData

from Persistence import insert

# Download data

In [3]:
def download_pdf(idLegislature, idPleno):
    url = f'http://www.congreso.es/public_oficiales/L{idLegislature}/CONG/DS/PL/DSCD-{idLegislature}-PL-{idPleno}.PDF'
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'data/diaries/DSCD-{idLegislature:02d}-PL-{idPleno:03d}.PDF', 'wb') as file:
            file.write(response.content)
        return url
    return None

In [5]:
consecutive_errors = 0
max_consecutive_errors = 50
for legislature in tqdm(range(10, 15)):
    consecutive_errors = 0
    for pleno in tqdm(range(1, 500)):
        path = download_pdf(legislature, pleno)
        if path:
            consecutive_errors = 0
        else: 
            consecutive_errors = consecutive_errors + 1
            if consecutive_errors > max_consecutive_errors:
                break

# Convert PDF to TXT

In [11]:
pdfsPath = 'data/diaries/'
txtPath = 'data/diaries-txts/'

files = [f for f in listdir(pdfsPath) if isfile(join(pdfsPath, f))]

process = 'ReadPdf/ReadPdf/bin/Release/net472/ReadPdf.exe'
for i in trange(len(files)):
    file = files[i]
    src = join(pdfsPath, file)
    dst = join(txtPath, file[:-4] + '.txt')
    call([process, src, dst])

# Transform txt to TextData

In [3]:
pathTxts = [join('./data/diaries-txts/', f) for f in listdir('./data/diaries-txts') if isfile(join('./data/diaries-txts/', f))]
data = []
for i in trange(len(pathTxts)):
    path = pathTxts[i]
    diary = Diary(path)
    
    debates = diary.get_debates()
    for d in range(len(debates)):
        debate = debates[d]
        
        speaches = debate.get_speaches()
        for s in range(len(speaches)):
            speach = speaches[s]
            debateTitle = debate.title.strip()
            if not debateTitle:
                debateTitle = diary.firstDayOrder
            td = TextData(speach.speach, {
                "celebrated": diary.celebrated,
                "legislature": diary.legislature,
                
                "debateOrder": d,
                "debate": debateTitle,
                
                "speachOrder": s,
                "speackerTreatment": speach.speackerTreatment,
                "speacker": speach.speacker,
                "path": path
            })
            
            if not diary.celebrated or not diary.legislature or not debateTitle or not speach.speackerTreatment or not speach.speacker:
                print(td)
            
            data.append(td)
        
    

# Insert into DB

```
docker build -t postgresql ./docker/postgresql
docker run -it --rm --name postgresql -p 5432:5432 -v D:/Sources/TFG/data/postgresql:/var/lib/postgresql/data postgresql
```

In [4]:
for i in trange(len(data)):
    insert(data[i])